# League of Legends API information and implementation

## Queue

To "queue up" in a game means to join a waiting list or a matchmaking system to play a game with other players. There are different types of queues:

Queue list = [RANKED_SOLO_5x5, RANKED_FLEX_SR, RANKED_FLEX_TT]

## Tier

Ranked system in League of Legends is divided into nine different tiers:

Tier list = [IRON, BRONZE, SILVER, GOLD, PLATINUM, DIAMOND, MASTER, GRANDMASTER, CHALLENGER]

## Division

Each of the nine tiers are further subdivided into divisions. For example, there are four divisions for each tier between Iron to Diamond.

From Master rank onwards, there is no sub-division

Division list = [I, II, III, IV]

## Regions

Riot Games provides servers to host various aspects of League of Legends icon League of Legends. These include but are not limited to logins, games, chat, the RP Store, the in-game shop, etc.

Region dict = {"BR1" : "Brazil", "EUN1" : "Europe Nordic & East", "EUW1" : "Europe West", "LA1" : "Latin America North", "LA2" : "Latin America South", "NA1" : "North America", "OCE" : "Oceania", "RU" : "Russia", "TR" : "Turkey", "JP" : "Japan", "KR" : "Republic of Korea", "PH" : "The Philippines", "SG" : "Singapore, Malaysia, & Indonesia", "TW" : "Taiwan, Hong Kong, and Macao", "TH" : "Thailand", "VN" : "Vietnam"}

### LEAGUE-EXP-V4

Link: https://developer.riotgames.com/apis#league-exp-v4

/lol/league-exp/v4/entries/{queue}/{tier}/{division}

Returns information of players who have played ranked and have a tier

Request URL example: https://la2.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/IV?page=2&api_key=RGAPI-64cb4b69-7271-4a1f-a919-acc0943d3129

Needed info: Queue, Tier, Division, Region, Page

### Requests to LEAGUE-EXP-V4

#### Needed Imports

In [3]:
#Import needed packages
import pandas as pd

import requests

#To limit the amount of requests per minute
from ratelimit import limits, sleep_and_retry #from https://pypi.org/project/ratelimit/

%load_ext autoreload
%autoreload 2

#### Needed Information

In [4]:
#Spirit of the LoL api key
api_key = "RGAPI-529e858b-1b62-4c41-a056-3e83cb1b796d"

#Code for LAS region
region = "LA2"

#Code for queue Solo
queue = "RANKED_SOLO_5x5"

queue_list = ["RANKED_SOLO_5x5", "RANKED_FLEX_SR", "RANKED_FLEX_TT"]

#Tiers
tier_list = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]

#Remember that from Master rank and above, only division I is valid
division_list = ["IV", "III", "II", "I"]

#### Needed Functions for league_exp_v4

In [3]:
def save_to_csv_league_exp_v4(df, queue):
    """
    Saves the dataframe obtained from league_exp_v4 into a csv file
    """
    
    df.to_csv(f'Data/league_exp_v4_{region}_{queue}.csv')  
    
    return print(f'Saved to Data/league_exp_v4_{region}_{queue}.csv')

In [4]:
#100 requests every 120 sec

time_seconds = 10

max_requests = 8

@sleep_and_retry
@limits(calls= max_requests, period= time_seconds) #period is in seconds 
def fetch_league_exp_v4(queue, tier, division, region, page, api_key):
    """
    Get LeagueEntryDTO from LEAGUE-EXP-V4 API
    """
    url = f'https://{region}.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}?page={page}&api_key={api_key}'
    
    response = requests.get(url)
    
    if response.status_code != 200:
                    raise Exception('API response: {}'.format(response.status_code))
            
    return response.json()

In [5]:
def iterate_league_exp_v4():
    """
    Get all LeagueEntryDTO from LEAGUE-EXP-V4 API for a certain tier.
    """
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Loops over the Tier list
    for tier in tier_list:
    
        #Verifies tier to define possible divisions
        if tier in ["MASTER", "GRANDMASTER", "CHALLENGER"]:
            division_list = ["I"]
        else:
            division_list = ["IV", "III", "II", "I"]

        #Loops over divisions
        for division in division_list:

            #Creates page counter
            page = 1 

            #Creates not empty DF
            response_df = pd.DataFrame({'not_empty': [1]})

            #Loops over pages until it finds and empty response
            while not response_df.empty:

                    response_df = pd.DataFrame(fetch_league_exp_v4(queue, tier, division, region, page, api_key))

                    complete_df = pd.concat([complete_df,response_df], ignore_index = True)

                    print(f"Tier: {tier}, Division: {division}, Page: {page}")

                    page = page + 1

                    if response_df.empty:
                        print("Empty")
                
    return complete_df

In [140]:
%%time

league_exp_v4_df2 = iterate_league_exp_v4()

Tier: IRON, Division: IV, Page: 1
Tier: IRON, Division: IV, Page: 2
Tier: IRON, Division: IV, Page: 3
Tier: IRON, Division: IV, Page: 4
Tier: IRON, Division: IV, Page: 5
Tier: IRON, Division: IV, Page: 6
Tier: IRON, Division: IV, Page: 7
Tier: IRON, Division: IV, Page: 8
Tier: IRON, Division: IV, Page: 9
Tier: IRON, Division: IV, Page: 10
Tier: IRON, Division: IV, Page: 11
Tier: IRON, Division: IV, Page: 12
Tier: IRON, Division: IV, Page: 13
Tier: IRON, Division: IV, Page: 14
Tier: IRON, Division: IV, Page: 15
Tier: IRON, Division: IV, Page: 16
Empty
Tier: IRON, Division: III, Page: 1
Tier: IRON, Division: III, Page: 2
Tier: IRON, Division: III, Page: 3
Tier: IRON, Division: III, Page: 4
Tier: IRON, Division: III, Page: 5
Tier: IRON, Division: III, Page: 6
Tier: IRON, Division: III, Page: 7
Tier: IRON, Division: III, Page: 8
Tier: IRON, Division: III, Page: 9
Tier: IRON, Division: III, Page: 10
Tier: IRON, Division: III, Page: 11
Tier: IRON, Division: III, Page: 12
Tier: IRON, Division

KeyboardInterrupt: 

In [137]:
save_to_csv_league_exp_v4(league_exp_v4_df, queue)

Saved to Data/league_exp_v4_LA2_RANKED_SOLO_5x5.csv


In [142]:
league_exp_v4_df

leagueId        queueType        tier  \
0       02d7ea50-acbe-456f-9585-71cd53337eb4  RANKED_SOLO_5x5        IRON   
1       945d44da-c2b5-4872-ac1e-feea379283c4  RANKED_SOLO_5x5        IRON   
2       5c3cde8c-c68e-4f09-8f90-227207aef4dc  RANKED_SOLO_5x5        IRON   
3       35f17e8d-96ea-4a6d-a17e-a4aaceb52e18  RANKED_SOLO_5x5        IRON   
4       596811f6-afd0-410a-aae3-78bdc6ed1ecf  RANKED_SOLO_5x5        IRON   
...                                      ...              ...         ...   
649250  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5  CHALLENGER   
649251  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5  CHALLENGER   
649252  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5  CHALLENGER   
649253  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5  CHALLENGER   
649254  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5  CHALLENGER   

       rank                                         summonerId   summonerName  \
0        IV    WuFzWmJf88LqcyAneH38fm_KTEvjMV-6jhm9v1XjPTZRhyA   brandonmoraa   
1        IV    egxYZxlTYh-ZpYkZu0nlJIBpnqM3MrCGyu_FyzP8hNA9YO4    mainzed2002   
2        IV    BLdBsYJH7CjtNZ0scTg-5-NOgb5hs6Hp-KOEw4R5ApacvCk      Don Napio   
3        IV    RW-C6essXemdYtFBdbacJcA_3-MVI7nZ0qywC-CoXzfBGa4     Trimitio18   
4        IV    6TiYs8moIVNuOtroXoo1v0Vrnl3ZZw0fV6GxmToFUzpRmTc  ZzZ DeathNote   
...     ...                                                ...            ...   
649250    I     _hreSKfWsC4Vp2M8m_F1V4a78yTphwJZQEAe7PcKYugDHw   gnhtbjriyk5u   
649251    I    gS92tgEasRwbZcQKS-obSauVQNwKEhYx6iodAwMjxCtmVvU        sincrom   
649252    I       Buiz5pmv7Z553fSGRlaMCMqKKYfmFY1uRh1fMdHKSjCO           svik   
649253    I       SR5ydVVF_Dq6mbP2vUKhTkaB7hnP398Doh0bter8K-ET    patacochina   
649254    I  vEX6KbVxuwvMOcgPgLUkwUMIpMUJFrUXszRw94XOBztbUD...  ƒø ßuggati øƒ   

        leaguePoints  wins  losses  veteran  inactive  freshBlood  hotStreak  \
0                 31    24      42    False     False       False      False   
1                  0    38      50    False     False       False      False   
2                 29    41      37    False     False       False      False   
3                 54   300     309    False     False       False      False   
4                 75    83     108    False     False       False      False   
...              ...   ...     ...      ...       ...         ...        ...   
649250           780   116      94    False     False        True      False   
649251           763    94      48    False     False       False      False   
649252           761   132     119    False     False       False      False   
649253           753   139     113    False     False        True      False   
649254           751   207     152    False     False       False      False   

       miniSeries  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
...           ...  
649250        NaN  
649251        NaN  
649252        NaN  
649253        NaN  
649254        NaN  

[649255 rows x 14 columns]

In [143]:
### Feature importance, decission tree

#### Needed Functions for summoner_v4

In [22]:
#1600 requests every 60 seconds
#or 100 requests every 120 sec? check

time_seconds = 10

max_requests = 8

@sleep_and_retry
@limits(calls= max_requests, period= time_seconds) #period is in seconds 
def fetch_summoner_v4(region, summoner_name, api_key):
    """
    Get SummonerDTO from SUMMONER_V4 API
    """
    url = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}"
    
    #try:
    
    response = requests.get(url)
        
    print(f"Summoner Name: {summoner_name}, 'API response: {response.status_code}")
        
   # except:
                        
        #print(f"Summoner Name: {summoner_name}, 'API response: {resonse.status_code}")
        
        #response = pd.DataFrame()
            
       # pass
    
    #if response.status_code != 200:
    #                raise Exception('API response: {}'.format(response.status_code))
            
    return response.json()

In [6]:
league_exp_v4_df = pd.read_csv("Data/league_exp_v4_LA2_RANKED_SOLO_5x5.csv")

league_exp_v4_df

/tmp/ipykernel_623/3287991397.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  league_exp_v4_df = pd.read_csv("Data/league_exp_v4_LA2_RANKED_SOLO_5x5.csv")


Unnamed: 0                              leagueId        queueType  \
0                0  02d7ea50-acbe-456f-9585-71cd53337eb4  RANKED_SOLO_5x5   
1                1  945d44da-c2b5-4872-ac1e-feea379283c4  RANKED_SOLO_5x5   
2                2  5c3cde8c-c68e-4f09-8f90-227207aef4dc  RANKED_SOLO_5x5   
3                3  35f17e8d-96ea-4a6d-a17e-a4aaceb52e18  RANKED_SOLO_5x5   
4                4  596811f6-afd0-410a-aae3-78bdc6ed1ecf  RANKED_SOLO_5x5   
...            ...                                   ...              ...   
649250      649250  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5   
649251      649251  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5   
649252      649252  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5   
649253      649253  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5   
649254      649254  2b0b12d1-e248-3f93-ad96-191715b57ee1  RANKED_SOLO_5x5   

              tier rank                                         summonerId  \
0             IRON   IV    WuFzWmJf88LqcyAneH38fm_KTEvjMV-6jhm9v1XjPTZRhyA   
1             IRON   IV    egxYZxlTYh-ZpYkZu0nlJIBpnqM3MrCGyu_FyzP8hNA9YO4   
2             IRON   IV    BLdBsYJH7CjtNZ0scTg-5-NOgb5hs6Hp-KOEw4R5ApacvCk   
3             IRON   IV    RW-C6essXemdYtFBdbacJcA_3-MVI7nZ0qywC-CoXzfBGa4   
4             IRON   IV    6TiYs8moIVNuOtroXoo1v0Vrnl3ZZw0fV6GxmToFUzpRmTc   
...            ...  ...                                                ...   
649250  CHALLENGER    I     _hreSKfWsC4Vp2M8m_F1V4a78yTphwJZQEAe7PcKYugDHw   
649251  CHALLENGER    I    gS92tgEasRwbZcQKS-obSauVQNwKEhYx6iodAwMjxCtmVvU   
649252  CHALLENGER    I       Buiz5pmv7Z553fSGRlaMCMqKKYfmFY1uRh1fMdHKSjCO   
649253  CHALLENGER    I       SR5ydVVF_Dq6mbP2vUKhTkaB7hnP398Doh0bter8K-ET   
649254  CHALLENGER    I  vEX6KbVxuwvMOcgPgLUkwUMIpMUJFrUXszRw94XOBztbUD...   

         summonerName  leaguePoints  wins  losses  veteran  inactive  \
0        brandonmoraa            31    24      42    False     False   
1         mainzed2002             0    38      50    False     False   
2           Don Napio            29    41      37    False     False   
3          Trimitio18            54   300     309    False     False   
4       ZzZ DeathNote            75    83     108    False     False   
...               ...           ...   ...     ...      ...       ...   
649250   gnhtbjriyk5u           780   116      94    False     False   
649251        sincrom           763    94      48    False     False   
649252           svik           761   132     119    False     False   
649253    patacochina           753   139     113    False     False   
649254  ƒø ßuggati øƒ           751   207     152    False     False   

        freshBlood  hotStreak miniSeries  
0            False      False        NaN  
1            False      False        NaN  
2            False      False        NaN  
3            False      False        NaN  
4            False      False        NaN  
...            ...        ...        ...  
649250        True      False        NaN  
649251       False      False        NaN  
649252       False      False        NaN  
649253        True      False        NaN  
649254       False      False        NaN  

[649255 rows x 15 columns]

In [58]:
league_exp_v4_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649255 entries, 0 to 649254
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    649255 non-null  int64 
 1   leagueId      649255 non-null  object
 2   queueType     649255 non-null  object
 3   tier          649255 non-null  object
 4   rank          649255 non-null  object
 5   summonerId    649255 non-null  object
 6   summonerName  649255 non-null  object
 7   leaguePoints  649255 non-null  int64 
 8   wins          649255 non-null  int64 
 9   losses        649255 non-null  int64 
 10  veteran       649255 non-null  bool  
 11  inactive      649255 non-null  bool  
 12  freshBlood    649255 non-null  bool  
 13  hotStreak     649255 non-null  bool  
 14  miniSeries    10713 non-null   object
dtypes: bool(4), int64(4), object(7)
memory usage: 57.0+ MB


In [16]:
len(league_exp_v4_df[league_exp_v4_df["tier"] == "CHALLENGER"])

200

In [17]:
league_exp_v4_df[league_exp_v4_df["tier"] == "CHALLENGER"]["summonerName"]

649055     Flower Garden
649056    THE GATEKÉEPER
649057         hARAkiRi1
649058        SANI AlluK
649059           Ustanak
               ...      
649250      gnhtbjriyk5u
649251           sincrom
649252              svik
649253       patacochina
649254     ƒø ßuggati øƒ
Name: summonerName, Length: 200, dtype: object

In [84]:
def iterate_summoner_v4(league_exp_v4_df):
    """
    Get all SummonerDTO from SUMMONER-V4 API, it takes a long time because of API limit
    """
    #Takes de series I need with the summoner name and converts it into an array to loop
    summoner_name_array = league_exp_v4_df["summonerName"].to_numpy()
    
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Inicio un contador en 0
    count = 0
    
    #Loops over summoner name
    for i in range(0,1000):
              
        response_df = pd.DataFrame((fetch_summoner_v4(region, summoner_name_array[i], api_key)),index=[0])

        complete_df = pd.concat([complete_df,response_df], ignore_index = True)
            
        count = count + 1
                    
        if count % 100 == 0:
            
            print(f"Count: {count}")#Prints when we send 100 requests)
                
    return complete_df

In [47]:
def iterate_summoner_v4_lite(league_exp_v4_df, tier, max_data):
    """
    Get max_data SummonerDTO from SUMMONER-V4 API
    """
    number_summoners= len(league_exp_v4_df[league_exp_v4_df["tier"] == tier])
    
    print(f"\nNumber of Summoners in tier {tier}: {number_summoners}\n")
    
    #Takes de series I need with the summoner name and converts it into an array to loop
    summoner_name_array = league_exp_v4_df[league_exp_v4_df["tier"] == tier]["summonerName"].to_numpy()
    
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Inicio un contador en 0
    count = 0
    
    #Loops over summoner name
    if number_summoners < max_data:
        for i in range(0,number_summoners):
            
            print(f"Summoner {i+1} from {number_summoners}")

            response_df = pd.DataFrame((fetch_summoner_v4(region, summoner_name_array[i], api_key)),index=[0])

            complete_df = pd.concat([complete_df,response_df], ignore_index = True)

            count = count + 1

            if count % 50 == 0:

                print(f"\nCount: {count}\n")#Prints when we send 100 requests)
    
    else:
        for i in range(0,number_summoners,int(number_summoners/max_data)):
            
            print(f"Summoner {i+1} from {number_summoners}")

            response_df = pd.DataFrame((fetch_summoner_v4(region, summoner_name_array[i], api_key)),index=[0])

            complete_df = pd.concat([complete_df,response_df], ignore_index = True)

            count = count + 1

            if count % 50 == 0:

                print(f"\nCount: {count}\n")#Prints when we send 100 requests)
                
    return complete_df

In [41]:
def loop_save_to_csv_summoner_v4_lite(df, tier):
    """
    Saves the dataframe obtained from summoner_v4_lite into a csv file
    """
    df.to_csv(f'Data/summoner_v4_lite_{tier}.csv')  
    
    return print(f'\nSaved to Data/summoner_v4_lite_{tier}.csv\n')

In [50]:
%%time

upper_tier_list = ["DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]

middle_tier_list = ["GOLD", "PLATINUM"]

bottom_tier_list = ["IRON", "BRONZE", "SILVER"]

max_data = 1000

for i in bottom_tier_list:
    df = iterate_summoner_v4_lite(league_exp_v4_df, i, max_data)
    loop_save_to_csv_summoner_v4_lite(df, i)



Number of Summoners in tier IRON: 40223

Summoner 1 from 40223
Summoner Name: brandonmoraa, 'API response: 200
Summoner 41 from 40223
Summoner Name: Xaspart, 'API response: 200
Summoner 81 from 40223
Summoner Name: ZetaleonpaezGame, 'API response: 200
Summoner 121 from 40223
Summoner Name: ETSecretariaS3X0, 'API response: 200
Summoner 161 from 40223
Summoner Name: Crialsoff, 'API response: 200
Summoner 201 from 40223
Summoner Name: RumblesxBoric, 'API response: 200
Summoner 241 from 40223
Summoner Name: havokdeath, 'API response: 200
Summoner 281 from 40223
Summoner Name: Aaron4565, 'API response: 200
Summoner 321 from 40223
Summoner Name: Brandr3, 'API response: 200
Summoner 361 from 40223
Summoner Name: wed024, 'API response: 200
Summoner 401 from 40223
Summoner Name: XxJhoël wasonxX, 'API response: 200
Summoner 441 from 40223
Summoner Name: Cristhianxd1456, 'API response: 200
Summoner 481 from 40223
Summoner Name: drinfa, 'API response: 200
Summoner 521 from 40223
Summoner Name: Es

Summoner Name: kirius666, 'API response: 200
Summoner 4641 from 40223
Summoner Name: kevinn420, 'API response: 200
Summoner 4681 from 40223
Summoner Name: saguil14, 'API response: 200
Summoner 4721 from 40223
Summoner Name: NoMeLaContainer, 'API response: 200
Summoner 4761 from 40223
Summoner Name: joacotv4, 'API response: 200
Summoner 4801 from 40223
Summoner Name: 1FUTURE1, 'API response: 200
Summoner 4841 from 40223
Summoner Name: MellowSwing, 'API response: 200
Summoner 4881 from 40223
Summoner Name: Ka0ss, 'API response: 200
Summoner 4921 from 40223
Summoner Name: MatuCanobb, 'API response: 200
Summoner 4961 from 40223
Summoner Name: PeriHellioN, 'API response: 200
Summoner 5001 from 40223
Summoner Name: de1untulaso, 'API response: 200
Summoner 5041 from 40223
Summoner Name: Eikadaj, 'API response: 200
Summoner 5081 from 40223
Summoner Name: MefiistO169, 'API response: 200
Summoner 5121 from 40223
Summoner Name: ElSantuso, 'API response: 200
Summoner 5161 from 40223
Summoner Name:

Summoner Name: joelsolagato1, 'API response: 200
Summoner 9281 from 40223
Summoner Name: XxVICTORxXXX, 'API response: 200
Summoner 9321 from 40223
Summoner Name: PetaK, 'API response: 200
Summoner 9361 from 40223
Summoner Name: demiansito, 'API response: 200
Summoner 9401 from 40223
Summoner Name: rocketman, 'API response: 200
Summoner 9441 from 40223
Summoner Name: yzz0f, 'API response: 200
Summoner 9481 from 40223
Summoner Name: SoldierOffHell, 'API response: 200
Summoner 9521 from 40223
Summoner Name: gerouru0923, 'API response: 200
Summoner 9561 from 40223
Summoner Name: marcelosalts, 'API response: 200
Summoner 9601 from 40223
Summoner Name: eydaankeifer, 'API response: 200
Summoner 9641 from 40223
Summoner Name: PAP Zeus, 'API response: 200
Summoner 9681 from 40223
Summoner Name: Elliwood, 'API response: 200
Summoner 9721 from 40223
Summoner Name: imBussky, 'API response: 200
Summoner 9761 from 40223
Summoner Name: Super noob, 'API response: 200
Summoner 9801 from 40223
Summoner 

Summoner Name: ZapasoEres, 'API response: 200
Summoner 13841 from 40223
Summoner Name: Cacha Canaya, 'API response: 200
Summoner 13881 from 40223
Summoner Name: BadNey, 'API response: 200
Summoner 13921 from 40223
Summoner Name: santinokz9813, 'API response: 200
Summoner 13961 from 40223
Summoner Name: TQAlejandroG647, 'API response: 200

Count: 350

Summoner 14001 from 40223
Summoner Name: xSev7n, 'API response: 200
Summoner 14041 from 40223
Summoner Name: Amasamela, 'API response: 200
Summoner 14081 from 40223
Summoner Name: AkOuTT47, 'API response: 200
Summoner 14121 from 40223
Summoner Name: ñoño666, 'API response: 200
Summoner 14161 from 40223
Summoner Name: w4rbobby, 'API response: 200
Summoner 14201 from 40223
Summoner Name: DGxZ, 'API response: 200
Summoner 14241 from 40223
Summoner Name: lautym00, 'API response: 200
Summoner 14281 from 40223
Summoner Name: thebrunio, 'API response: 200
Summoner 14321 from 40223
Summoner Name: JOSE MERCADO, 'API response: 200
Summoner 14361 fro

Summoner Name: Letalidad22, 'API response: 200
Summoner 18401 from 40223
Summoner Name: ELGORDODRILLL, 'API response: 200
Summoner 18441 from 40223
Summoner Name: xLucasxCapitax, 'API response: 200
Summoner 18481 from 40223
Summoner Name: Aysel, 'API response: 200
Summoner 18521 from 40223
Summoner Name: EN  CULO, 'API response: 200
Summoner 18561 from 40223
Summoner Name: UnPresoDeJujuy, 'API response: 200
Summoner 18601 from 40223
Summoner Name: DURIZAR, 'API response: 200
Summoner 18641 from 40223
Summoner Name: Droguita420, 'API response: 200
Summoner 18681 from 40223
Summoner Name: mcQIYANA, 'API response: 200
Summoner 18721 from 40223
Summoner Name: diegoskar, 'API response: 200
Summoner 18761 from 40223
Summoner Name: BenjaMataHackers, 'API response: 200
Summoner 18801 from 40223
Summoner Name: Rodrix2374, 'API response: 200
Summoner 18841 from 40223
Summoner Name: DarkDemon516B, 'API response: 200
Summoner 18881 from 40223
Summoner Name: bamsxone, 'API response: 200
Summoner 18

...

Summoner Name: PorSiempreHierro, 'API response: 200
Summoner 91763 from 194785
Summoner Name: miguexi, 'API response: 200
Summoner 91957 from 194785
Summoner Name: LeoUbillus, 'API response: 200
Summoner 92151 from 194785
Summoner Name: Leanly, 'API response: 200
Summoner 92345 from 194785
Summoner Name: Mastershyfu, 'API response: 200
Summoner 92539 from 194785
Summoner Name: AlbertoFernández, 'API response: 200
Summoner 92733 from 194785
Summoner Name: GengisSW, 'API response: 200
Summoner 92927 from 194785
Summoner Name: Ourienaor, 'API response: 200
Summoner 93121 from 194785
Summoner Name: Malole, 'API response: 200
Summoner 93315 from 194785
Summoner Name: StrongTotipret, 'API response: 200
Summoner 93509 from 194785
Summoner Name: Ivie, 'API response: 200
Summoner 93703 from 194785
Summoner Name: Assasinxd23, 'API response: 200
Summoner 93897 from 194785
Summoner Name: ZYKKEK ADC, 'API response: 200
Summoner 94091 from 194785
Summoner Name: ll Yuna ll, 'API response: 200
Summone

Summoner Name: Roocioo, 'API response: 200
Summoner 113491 from 194785
Summoner Name: TitoYatusae, 'API response: 200
Summoner 113685 from 194785
Summoner Name: Arcabuz, 'API response: 200
Summoner 113879 from 194785
Summoner Name: GUARDA CON TOBI, 'API response: 200
Summoner 114073 from 194785
Summoner Name: romantiic, 'API response: 200
Summoner 114267 from 194785
Summoner Name: xX Leeyends Xx, 'API response: 200
Summoner 114461 from 194785
Summoner Name: CjvsCL, 'API response: 200
Summoner 114655 from 194785
Summoner Name: AlfreMontesDeOca, 'API response: 200
Summoner 114849 from 194785
Summoner Name: clau27, 'API response: 200
Summoner 115043 from 194785
Summoner Name: voltronuwu, 'API response: 200
Summoner 115237 from 194785
Summoner Name: Z3ERØ, 'API response: 200
Summoner 115431 from 194785
Summoner Name: cualquierad, 'API response: 200
Summoner 115625 from 194785
Summoner Name: Yato Sensei, 'API response: 200
Summoner 115819 from 194785
Summoner Name: conmor, 'API response: 20

Summoner Name: ramonalamonax, 'API response: 200
Summoner 135025 from 194785
Summoner Name: Loboarhe, 'API response: 200
Summoner 135219 from 194785
Summoner Name: camkz, 'API response: 200
Summoner 135413 from 194785
Summoner Name: Y4METE KUD4SS4I, 'API response: 200
Summoner 135607 from 194785
Summoner Name: Yoyito0, 'API response: 200

Count: 700

Summoner 135801 from 194785
Summoner Name: XeVeX, 'API response: 200
Summoner 135995 from 194785
Summoner Name: TP J o b s, 'API response: 200
Summoner 136189 from 194785
Summoner Name: Cry Me A Rıven, 'API response: 200
Summoner 136383 from 194785
Summoner Name: FurryKrissMain, 'API response: 200
Summoner 136577 from 194785
Summoner Name: Â R W E N, 'API response: 200
Summoner 136771 from 194785
Summoner Name: Nachinh0, 'API response: 200
Summoner 136965 from 194785
Summoner Name: 1Manu1, 'API response: 200
Summoner 137159 from 194785
Summoner Name: nojom, 'API response: 200
Summoner 137353 from 194785
Summoner Name: KashmirInnuendo, 'API

Summoner Name: Desotto, 'API response: 200
Summoner 156559 from 194785
Summoner Name: Leeme, 'API response: 200
Summoner 156753 from 194785
Summoner Name: Elesh Norn, 'API response: 200
Summoner 156947 from 194785
Summoner Name: El Messiratatata, 'API response: 404
Summoner 157141 from 194785
Summoner Name: SG Mado, 'API response: 200
Summoner 157335 from 194785
Summoner Name: bbynose, 'API response: 200
Summoner 157529 from 194785
Summoner Name: Gvtøxic, 'API response: 200
Summoner 157723 from 194785
Summoner Name: El GuRu, 'API response: 200
Summoner 157917 from 194785
Summoner Name: Sµndrømę, 'API response: 200
Summoner 158111 from 194785
Summoner Name: GinhowXDe, 'API response: 200
Summoner 158305 from 194785
Summoner Name: THE NEME, 'API response: 200
Summoner 158499 from 194785
Summoner Name: PumaRodriguez44, 'API response: 200
Summoner 158693 from 194785
Summoner Name: gordasusana, 'API response: 200
Summoner 158887 from 194785
Summoner Name: sSavage, 'API response: 200
Summoner

Summoner Name: rojasus, 'API response: 200
Summoner 178093 from 194785
Summoner Name: R7 KĄ ZHÎX, 'API response: 200
Summoner 178287 from 194785
Summoner Name: yukitoki, 'API response: 200
Summoner 178481 from 194785
Summoner Name: Cngointhu, 'API response: 200
Summoner 178675 from 194785
Summoner Name: Listahda, 'API response: 200
Summoner 178869 from 194785
Summoner Name: Yo soy EL, 'API response: 200
Summoner 179063 from 194785
Summoner Name: Crive55, 'API response: 200
Summoner 179257 from 194785
Summoner Name: StepFears1, 'API response: 200
Summoner 179451 from 194785
Summoner Name: Goku Fernandez, 'API response: 200
Summoner 179645 from 194785
Summoner Name: Zorrito126A, 'API response: 200
Summoner 179839 from 194785
Summoner Name: Aleph Zer0, 'API response: 200
Summoner 180033 from 194785
Summoner Name: T1 Valekey, 'API response: 200
Summoner 180227 from 194785
Summoner Name: BoB Esp0nja, 'API response: 200
Summoner 180421 from 194785
Summoner Name: TheSum41, 'API response: 200


In [85]:
%%time

summoner_v4_df = iterate_summoner_v4(league_exp_v4_df)

Summoner Name: brandonmoraa, 'API response: 200
Count: 1
Summoner Name: mainzed2002, 'API response: 200
Count: 2
Summoner Name: Don Napio, 'API response: 200
Count: 3
Summoner Name: Trimitio18, 'API response: 200
Count: 4
Summoner Name: ZzZ DeathNote, 'API response: 200
Count: 5
Summoner Name: RuleteYuY, 'API response: 200
Count: 6
Summoner Name: m0chiipochi, 'API response: 200
Count: 7
Summoner Name: Yaircito, 'API response: 200
Count: 8
Summoner Name: sigmalpike, 'API response: 200
Count: 9
Summoner Name: almafuerte39, 'API response: 200
Count: 10
Summoner Name: Øso Rüsso xnx, 'API response: 200
Count: 11
Summoner Name: MerkaSaurio, 'API response: 200
Count: 12
Summoner Name: Kreymax, 'API response: 200
Count: 13
Summoner Name: nochee, 'API response: 200
Count: 14
Summoner Name: CheverlyWix, 'API response: 200
Count: 15
Summoner Name: BertaBurte, 'API response: 200
Count: 16
Summoner Name: Gonchix, 'API response: 200
Count: 17
Summoner Name: saddam999, 'API response: 200
Count: 18
Su

Summoner Name: Ian LG, 'API response: 200
Count: 148
Summoner Name: GRUFFY, 'API response: 200
Count: 149
Summoner Name: nikito3252, 'API response: 200
Count: 150
Summoner Name: RitterOficial, 'API response: 200
Count: 151
Summoner Name: KINGHT, 'API response: 200
Count: 152
Summoner Name: gino9, 'API response: 200
Count: 153
Summoner Name: Ge114to, 'API response: 200
Count: 154
Summoner Name: animatoonpro057, 'API response: 200
Count: 155
Summoner Name: Igrc06, 'API response: 200
Count: 156
Summoner Name: lXxJuniorxXl, 'API response: 200
Count: 157
Summoner Name: Naufomi Kun, 'API response: 200
Count: 158
Summoner Name: Yussef, 'API response: 200
Count: 159
Summoner Name: TheUlTimaTeX1, 'API response: 200
Count: 160
Summoner Name: Crialsoff, 'API response: 200
Count: 161
Summoner Name: alexmodzzzz, 'API response: 200
Count: 162
Summoner Name: EMD GreenSmite, 'API response: 200
Count: 163
Summoner Name: masy, 'API response: 200
Count: 164
Summoner Name: nnvnt, 'API response: 200
Count:

Summoner Name: ByRaul23, 'API response: 200
Count: 294
Summoner Name: LautaroPugliese, 'API response: 200
Count: 295
Summoner Name: Zaneti643, 'API response: 200
Count: 296
Summoner Name: Mike Mosto, 'API response: 200
Count: 297
Summoner Name: ÐąrkÐąÿš, 'API response: 200
Count: 298
Summoner Name: InspiredLinx455, 'API response: 200
Count: 299
Summoner Name: jjacob, 'API response: 200
Count: 300
Summoner Name: Littlehotdog, 'API response: 200
Count: 301
Summoner Name: GaabriielSj, 'API response: 200
Count: 302
Summoner Name: Prendete S, 'API response: 200
Count: 303
Summoner Name: yesi215, 'API response: 200
Count: 304
Summoner Name: frxncii, 'API response: 200
Count: 305
Summoner Name: Dxer, 'API response: 200
Count: 306
Summoner Name: ElMasterPlay1, 'API response: 200
Count: 307
Summoner Name: TroowaA, 'API response: 200
Count: 308
Summoner Name: Ebcts17d, 'API response: 200
Count: 309
Summoner Name: Paceño, 'API response: 200
Count: 310
Summoner Name: Moisesconnor1, 'API response: 

Summoner Name: N4r4nj1t0, 'API response: 200
Count: 437
Summoner Name: XxGatitoGamerxX, 'API response: 200
Count: 438
Summoner Name: MilanesaEp, 'API response: 200
Count: 439
Summoner Name: Reshyan, 'API response: 200
Count: 440
Summoner Name: Cristhianxd1456, 'API response: 200
Count: 441
Summoner Name: Alinassincar, 'API response: 200
Count: 442
Summoner Name: GOOLEN, 'API response: 200
Count: 443
Summoner Name: Sc4rF4cE, 'API response: 200
Count: 444
Summoner Name: patalin1119, 'API response: 200
Count: 445
Summoner Name: SonaFortune, 'API response: 200
Count: 446
Summoner Name: ElBruniTroll, 'API response: 200
Count: 447
Summoner Name: ElBorrachoATR, 'API response: 200
Count: 448
Summoner Name: GastGS7, 'API response: 200
Count: 449
Summoner Name: Gallina 9 12, 'API response: 200
Count: 450
Summoner Name: BadSteel, 'API response: 200
Count: 451
Summoner Name: alansee, 'API response: 200
Count: 452
Summoner Name: Masela, 'API response: 200
Count: 453
Summoner Name: JanoXFX, 'API res

Summoner Name: DarkJuchus, 'API response: 200
Count: 582
Summoner Name: MaiShiranui, 'API response: 200
Count: 583
Summoner Name: Thexperience, 'API response: 200
Count: 584
Summoner Name: FireRanger21, 'API response: 200
Count: 585
Summoner Name: Lauti Ares, 'API response: 200
Count: 586
Summoner Name: culea gatos, 'API response: 200
Count: 587
Summoner Name: Keeviin1993DC, 'API response: 200
Count: 588
Summoner Name: balari, 'API response: 200
Count: 589
Summoner Name: xXjoaquinXx626, 'API response: 200
Count: 590
Summoner Name: kikiriki, 'API response: 200
Count: 591
Summoner Name: Armand Helsing, 'API response: 200
Count: 592
Summoner Name: Unknown YT, 'API response: 200
Count: 593
Summoner Name: PEPINIYO, 'API response: 200
Count: 594
Summoner Name: Dior SA, 'API response: 200
Count: 595
Summoner Name: DarkWake, 'API response: 200
Count: 596
Summoner Name: DanielZannln, 'API response: 200
Count: 597
Summoner Name: GordoOctogenario, 'API response: 200
Count: 598
Summoner Name: SH N

Summoner Name: brandylove07, 'API response: 200
Count: 727
Summoner Name: Bellota11, 'API response: 200
Count: 728
Summoner Name: SK0LD30, 'API response: 200
Count: 729
Summoner Name: MitsubishiEVOIII, 'API response: 200
Count: 730
Summoner Name: Metafisica, 'API response: 200
Count: 731
Summoner Name: Pszo, 'API response: 200
Count: 732
Summoner Name: Norrwing, 'API response: 200
Count: 733
Summoner Name: fasista2, 'API response: 200
Count: 734
Summoner Name: Mauro2026, 'API response: 200
Count: 735
Summoner Name: NahueCnts, 'API response: 200
Count: 736
Summoner Name: vladk10, 'API response: 200
Count: 737
Summoner Name: enzo1234ferrari, 'API response: 200
Count: 738
Summoner Name: AgusV8, 'API response: 200
Count: 739
Summoner Name: zTBlacKBlizarDTz, 'API response: 200
Count: 740
Summoner Name: Rompecatre2000, 'API response: 200
Count: 741
Summoner Name: BruXol8, 'API response: 200
Count: 742
Summoner Name: Danielcito lindo, 'API response: 200
Count: 743
Summoner Name: eltatauy, 'AP

Summoner Name: suciocrepi, 'API response: 200
Count: 872
Summoner Name: Aragon3, 'API response: 200
Count: 873
Summoner Name: Laureane, 'API response: 200
Count: 874
Summoner Name: nomada50, 'API response: 200
Count: 875
Summoner Name: rubilax2003, 'API response: 200
Count: 876
Summoner Name: 7omAX21, 'API response: 200
Count: 877
Summoner Name: Zevb, 'API response: 200
Count: 878
Summoner Name: piiiipe, 'API response: 200
Count: 879
Summoner Name: MonkesT PK, 'API response: 200
Count: 880
Summoner Name: Kløüz, 'API response: 200
Count: 881
Summoner Name: EVAN KING, 'API response: 200
Count: 882
Summoner Name: Anauhiraamm, 'API response: 200
Count: 883
Summoner Name: GuilleCRD, 'API response: 200
Count: 884
Summoner Name: spectro2811, 'API response: 200
Count: 885
Summoner Name: IronMaster69, 'API response: 200
Count: 886
Summoner Name: r3nz0gm, 'API response: 200
Count: 887
Summoner Name: YunguerFox, 'API response: 200
Count: 888
Summoner Name: Nicoardss, 'API response: 200
Count: 889

In [28]:
1001 % 1000

1

In [83]:
summoner_v4_df.puuid[1]

'7irsZb-qxVzSWRaPBr2HugJRXOeI7wqTfnbx_zltxX-iGipMknpJN1mMAQK4DJzP0u-BFGGcDJd15w'

In [86]:
summoner_v4_df.to_csv(f'Data/summoner_v4_1000.csv')  

#### Needed Functions for MATCH_V5

##### /lol/match/v5/matches/by-puuid/{puuid}/ids

Response Game IDs for summoner puuid

##### /lol/match/v5/matches/{matchId}

Response Game data for matchid

##### /lol/match/v5/matches/{matchId}/timeline

Response Game time data for matchid

In [5]:
tier_summoner_v4_lite = "CHALLENGER"

summoner_v4_lite_df = pd.read_csv(f"Data/summoner_v4_lite_{tier_summoner_v4_lite}.csv")

In [8]:
summoner_v4_lite_df["puuid"]

0      dSepgMmF0eZ7bPL00JU7jyO_MGJE97ceLEI3qRB_fRvptw...
1      rdV8DH_CLQDSQOrgf0M15wrLtMkFk7HXKyGCF0IXpeHLg5...
2      OeJyhbXo-UpMu6QTtcLXw78vc-_3KZ8L1BS78VR3ddIz9l...
3      zX9l5v58D9aTA1gSDc0nANPr1OQAHO9rWJsaI5qGG5IcE_...
4      fC8wcF9hEt5XmndrkHWGabMlwxRM4dRFpwrLW8WhvV5oor...
                             ...                        
195    sII-3_I1eMKOQ1TQo5h0CGy8rX6KG5YgTZSp_5YY6UHA19...
196    4PTK3-iZxL80x4cMwamm2GHfCk9o77zP8___iMuHKWosZP...
197    ZRsoSCxmwsYXQpbTCoWeJ0uqcxZ7W6pNW8KxBiubgJFY4Y...
198    CG2ULYnHwUtLsi93jsAmK76-uFDPGPGmpOEUGwDPqhAceo...
199    UDxi8HVv2Eux5ftTwUOguQylu7A4V1ckyJmtWl7HRj3vjE...
Name: puuid, Length: 200, dtype: object

In [15]:
#100 requests every 120 sec

time_seconds = 10

max_requests = 8

"""Parameters
puuid : unique globally IDs
api_key

type: Filter the list of match ids by the type of match. This filter 
is mutually inclusive of the queue filter meaning 
any match ids returned must match both the queue and type filters.

count: Number of match ids to return.

"""

@sleep_and_retry
@limits(calls= max_requests, period= time_seconds) #period is in seconds 
def fetch_match_V5(puuid, api_key, count = 100, match_type = "ranked"):
    """
    Get Match IDs from Match_V5
    """
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?&count={count}&type={match_type}&api_key={api_key}'
    
    response = requests.get(url)
    
    if response.status_code != 200:
                    raise Exception('API response: {}'.format(response.status_code))
            
    return response.json()

In [19]:
def iterate_match_V5(summoner_v4_lite_df):
    """
    Get all MatchID from Match_V5 API
    """
    #Takes de series I need with the summoner name and converts it into an array to loop
    puuid_name_array = summoner_v4_lite_df["puuid"].to_numpy()
    
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Inicio un contador en 0
    count = 0
    
    #Loops over summoner name
    for i in range(0,10):
              
        response_df = pd.DataFrame((fetch_match_V5(puuid_name_array[i], api_key)))

        complete_df = pd.concat([complete_df,response_df], ignore_index = True)
            
        print(f"Count: {count}")
        
        count = count + 1
                    
        if count % 100 == 0:
            
            print(f"Checkpoint!")#Prints when we send 100 requests)
                
    return complete_df

In [20]:
test = iterate_match_V5(summoner_v4_lite_df)

Count: 0
Count: 1
Count: 2
Count: 3
Count: 4
Count: 5
Count: 6
Count: 7
Count: 8
Count: 9


In [21]:
test

0
0    LA2_1299222804
1    LA2_1299195310
2    LA2_1299087524
3    LA2_1299072076
4    LA2_1299057860
..              ...
995  LA2_1297036337
996  LA2_1297030957
997  LA2_1297024575
998  LA2_1297016304
999  LA2_1297000744

[1000 rows x 1 columns]